DataModule类有一个父类(HyperParameters)

在DataModule类定义中，主要有以下方法

- `__init__` 

传入数据的url或者数据的根目录文件。downloading the data and preprocessing the data.

亦可以自己定义数据集(但仍然建议这一步在DIYDataset类中完成)
```python
self.X = torch.randn(n, len(w))
noise = torch.randn(n, 1) * noise
self.y = torch.matmul(self.X, w.reshape((-1, 1))) + b + noise 
```
- `def train_dataloader` 调用`def get_dataloader`并传入train==True，返回train的DataLoader。returns data loader for the training set.
- `def val_dataloader(option)` 调用`def get_dataloader`并传入train==False，返回validation的DataLoader。returns data loader for the validaton set.
- `def get_dataloader` 调用`def get_tensorloader`并传入是否是训练集，返回DataLoader
- `def get_tensorloader` 根据train标记生成对应dataset=Dataset的子类DIYDataset，并根据Dataset生成DataLoader，返回DataLoader

数据究竟如何`__getitem__`和`__len__`取决于DIYDataset类。

Module类有两个父类(torch.nn.Module, HyperParameters)

在Module类模型定义中，主要有以下几个方法

- `def __init__`: 
    
    定义模型参数，self.w, self.b或者self.net; 
    
    初始化模型参数; 
    
    生成绘制loss的图片，实例化ProgressBoard()类==self.board
- `def forward`:  模型参数和数据的计算方式，如何对参数进行训练
- `def loss`:     损失函数。传入signal_hat和target计算并返回loss。
- `def configure_optimizers`: 书写优化函数，作用在于如何使用算法使得loss最小，且更新参数。
- `def training_step` 在训练阶段接收一个batch的数据，这里的batch是已经训练好的batch，并将batch数据分为signal_hat和target传入`def loss`，计算两者的loss并返回，同时使用plot传入这个batch中的loss值。accepts a data batch to return the loss value.
- `def validation_step(option)` 测试阶段使用。evaluation measures.
- `def plot` 绘制一个epoch-loss曲线，但是会有一个参数叫做`self.plot_train_per_epoch`决定了一个epoch中有几个点会在图上绘制。

Trainer类有一个父类(HyperParameters)

在Trainer类定义中，主要有以下几个方法

- `def __init__` 初始化一些训练参数。

指定训练过程中的max_epochs, 

使用哪个GPU训练num_gpus,

 梯度是否裁剪以及对应裁剪值gradient_clip_val
- `def fit` 传入Module的实例model，DataModule实例data，进行训练。

调用`def prepare_data`

调用`def prepare_model`

调用`model.configure_optimizers`指定self.optim。使用module中的类方法返回值为trainer中的优化算法

初始化self.epoch, self.train_batch_idx, self.val_batch_idx

每一个epoch调用`def fit_epoch()`

- `def prepare_data` 传入data，即DataModule实例类。预处理在DIYDataset中或者更早已经完成。使用self.train_dataloader = data.train_dataloader()来给trainer的train_dataloader赋值。
- `def prepare_model` 传入Module实例类。指定trainer.model=model, model.trainer=trainer, model.board.xlim为trainer.max_epochs
- `def fit_epoch` 一个epoch所要做的事情
